In [ ]:
# general
import json
import collections
import math
import uuid
import random
import pickle
import os
from itertools import islice
import numpy as np
from tqdm import tqdm_notebook as tqdm

from src.geometry import sq_distance, vector_mag


In [ ]:
data_path = "/mnt/c/data/3D_CAD/"
dist_threshold = 0.05 # distance threshold for pipe proximity
centerline_angle_threshold = math.radians(10) # angle threshold for two pipes to be considered parallel
centerline_dist_threshold = 0.02 # centerline distance threshold pipes to be to considered connected

In [ ]:
# load data
with open(data_path + 'edges_westdeckbox.pkl', 'rb') as f:
    edges = pickle.load(f)
with open(data_path + 'nodes_westdeckbox.pkl', 'rb') as f:
    node_info = pickle.load(f)
    nodes = node_info[0]
    


In [ ]:
print(nodes[0], edges[0])

In [ ]:
e1 = [e[0] for e in edges]
e2 = [e[1] for e in edges]
print(max(e1), max(e2))
print(len(nodes))
print(len(edges))

In [ ]:
print(type(nodes[0][3]))

### this is adding entire pipes to the nearby edges list, all the code should be modified to specify which edge is being added, as the checks only run on edges not pipes.

In [ ]:
# find close-by edges
def edge_proximity_criteria(e1, e2, threshold):
    if ((sq_distance(e1[0][0], e1[0][1], e1[0][2], e2[0][0], e2[0][1], e2[0][2]) < threshold) or 
        (sq_distance(e1[1][0], e1[1][1], e1[1][2], e2[0][0], e2[0][1], e2[0][2]) < threshold) or 
        (sq_distance(e1[0][0], e1[0][1], e1[0][2], e2[1][0], e2[1][1], e2[1][2]) < threshold) or 
        (sq_distance(e1[1][0], e1[1][1], e1[1][2], e2[1][0], e2[0][1], e2[1][2]) < threshold)):
        return True

In [ ]:
# get pipe neighbours

# find edges of pipes
pipes = [[n[0], np.array(n[1]), np.array(n[2]), np.array(n[3]), n[4]] for n in nodes if n[0]==3]
pipe_edges = [(n[1] - (max(n[2])/2 * n[3]),
               n[1] + (max(n[2])/2 * n[3])) for n in pipes]
# print(len(pipe_edges))
# print(pipe_edges[0])

# compare edges to find close-by edges
nearby_edges = []
for i, pe1 in tqdm(enumerate(pipe_edges)):
    for j, pe2 in enumerate(pipe_edges[i+1:]):
        if edge_proximity_criteria(pe1, pe2, dist_threshold):
            nearby_edges.append((i,j+i+1))
            
print(len(nearby_edges), nearby_edges[0])

In [ ]:
# rough check for tees
possible_tees = []
for i, ne1 in tqdm(enumerate(nearby_edges)):
    for j, ne2 in enumerate(nearby_edges[i+1:]):
        if ((ne1[0] == ne2[0]) or 
            (ne1[1] == ne2[0]) or 
            (ne1[0] == ne2[1]) or 
            (ne1[1] == ne2[1])):
            s = {ne1[0], ne1[1], ne2[0], ne2[1]}
            possible_tees.append(list(s))
            
print(len(possible_tees), possible_tees[0])

In [ ]:
def pipe_check(a, b):
    # centerline direction check -
    centerline_deviation = math.acos(np.dot(a[3], b[3]))
    if centerline_deviation < centerline_angle_threshold:
        return True
    
def elbow_check(a, b):
    # centerline proximity check
    centerline_connecting_line = np.cross(a[3], b[3])
    edge_connecting_line = b[1] - a[1]
    centerline_distance = (math.abs(np.dot(centerline_connecting_line, 
                                          edge_connecting_line)) / 
                           vector_mag(centerline_connecting_line))

    if centerline_distance > centerline_dist_threshold:
        return True

In [ ]:
# check for tees
# if two of the elements seem to be connected in a straight line and the other is placed like an elbow, its considered a tee
tee_connections = []
for i, pt in enumerate(possible_tees):
    pipe_connection = False
    if pipe_check(pipe_edges[pt[0]], pipe_edges[pt[1]]):
        pipe_connection = True
        pipe_pair = [0,1]
        other = 2
    if pipe_check(pipe_edges[pt[0]], pipe_edges[pt[2]]):
        pipe_connection = True
        pipe_pair = [0,2]
        other = 1
    if pipe_check(pipe_edges[pt[1]], pipe_edges[pt[2]]):
        pipe_connection = True
        pipe_pair = [1,2]
        other = 0
        
    if pipe_connection:
        if elbow_check(pipe_edges[pt[pipe_pair[0]]], pipe_edges[pt[other]]):
            tee_connections.append(pt)
        

In [ ]:
# check for elbows or connected pipes
pipe_connections = []
elbow_connections = []
for i, ne in enumerate(nearby_edges): 

    if pipe_check(ne[0], ne[1]):
        pipe_connections.append(ne)
        continue
    
    if elbow_check(ne[0], ne[1]):
        elbow_connections.append(ne)
            
        






# remove detected tees

In [ ]:
# check for elbows
